## DQ Representation for Cyton Arm from DH

In [1]:
from DQ import *
from DQ_kinematics import *
import numpy as np
import scipy.linalg as LA
from math import pi

def DQ_CYTON():
    '''
    DQ Representation for COMAU Arm
    '''
    
    cyton_DH_theta=  np.array([0, 0, 0, 0, pi, pi/2, 0])
    cyton_DH_d =     np.array([0.0379, -0.0046, 0.145, -0.011, 0.175, 0.0074, 0.0])
    cyton_DH_a =     np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0677])
    cyton_DH_alpha = np.array([0, pi/2, -pi/2, pi/2, -pi/2, pi/2, pi/2])
    cyton_dummy =    np.array([0,0,0,0,0,0,0])

    cyton_DH_matrix = np.array([cyton_DH_theta,cyton_DH_d,cyton_DH_a,cyton_DH_alpha,cyton_dummy])

    cyton = DQ_kinematics(cyton_DH_matrix, 'modified')

    return cyton

## Open Loop Control Simulation

In [2]:
robotarm_kine = DQ_CYTON();

# Basic definitions for the simulation
theta  = np.array([0,0,-pi/2,0,pi/2,0,0])
thetad = np.array([pi/2,0,-pi/2,0,pi/2,0,0])

# Desired end-effector pose
xd = robotarm_kine.fkm(thetad);

error = 1;
epsilon = 0.001;
K = 0.5;

while LA.norm(error) > epsilon:
    x = robotarm_kine.fkm(theta);
    J = robotarm_kine.jacobian(theta);
    error = vec8(xd-x);
    theta = theta + np.dot(np.dot(LA.pinv(J),K),error);
    print LA.norm(error)

0.782529172168
1.3119865458
1.08703292083
0.675011182588
0.38063384537
0.201940598604
0.107061773686
0.0574885111265
0.031324435465
0.0174591895213
0.0101612863653
0.00643709787186
0.00474493952727
0.00436589985638
0.00521978010093
0.00782097500285
0.00868096332387
0.00481146296309
0.00246832919243
0.00124439555779
0.000623597051812


## Open Loop Control Using ROS

In [3]:
import rospy
from sensor_msgs.msg import JointState
from std_msgs.msg import Float32MultiArray
import numpy as np

robotarm_kine = DQ_CYTON();

# Basic definitions for the execution
theta  = np.array([0,0,-pi/2,0,pi/2,0,0])
thetad = np.array([pi/2,0,-pi/2,0,pi/2,0,0])

# Desired end-effector pose
xd = robotarm_kine.fkm(thetad);

error = 1;
epsilon = 0.001;
K = 0.5;

rospy.init_node('cyton_feedback',anonymous=False)
pub = rospy.Publisher('/Cyton/jointCmd', Float32MultiArray, queue_size=10)
rate = rospy.Rate(10) # 10hz

while LA.norm(error) > epsilon:
    x = robotarm_kine.fkm(theta);
    J = robotarm_kine.jacobian(theta);
    error = vec8(xd-x);
    theta = theta + np.dot(np.dot(LA.pinv(J),K),error);
    
    cmd_msg = Float32MultiArray()
    cmd_msg.data = theta
    pub.publish(cmd_msg)
    
    rospy.loginfo(LA.norm(error))
    rate.sleep()


[INFO] [WallTime: 1502833880.133564] 0.782529172168
[INFO] [WallTime: 1502833880.233439] 1.3119865458
[INFO] [WallTime: 1502833880.333682] 1.08703292083
[INFO] [WallTime: 1502833880.433269] 0.675011182588
[INFO] [WallTime: 1502833880.533491] 0.38063384537
[INFO] [WallTime: 1502833880.634394] 0.201940598604
[INFO] [WallTime: 1502833880.733675] 0.107061773686
[INFO] [WallTime: 1502833880.835532] 0.0574885111265
[INFO] [WallTime: 1502833880.934048] 0.031324435465
[INFO] [WallTime: 1502833881.033740] 0.0174591895213
[INFO] [WallTime: 1502833881.134059] 0.0101612863653
[INFO] [WallTime: 1502833881.233883] 0.00643709787186
[INFO] [WallTime: 1502833881.333772] 0.00474493952727
[INFO] [WallTime: 1502833881.433344] 0.00436589985638
[INFO] [WallTime: 1502833881.533679] 0.00521978010093
[INFO] [WallTime: 1502833881.633497] 0.00782097500285
[INFO] [WallTime: 1502833881.733486] 0.00868096332387
[INFO] [WallTime: 1502833881.833478] 0.00481146296309
[INFO] [WallTime: 1502833881.933594] 0.002468329192